In [72]:
import pandas as pd
from lxml import etree
import sys

In [73]:
df = pd.read_csv("../app/fsimage2402.csv")

In [69]:
col_index = list(range(0, len(df.columns)))
for name in ["NSQUOTA", "DSQUOTA"]:
    col_index.pop(list(df.columns).index("NSQUOTA"))

df.iloc[:, col_index]

,Path,Replication,ModificationTime,AccessTime,PreferredBlockSize,BlocksCount,FileSize,Permission,UserName,GroupName
0,/,0,2025-02-05 23:20,1970-01-01 0:00,0,0,0,drwxr-xr-x,ubuntu,supergroup
1,/home,0,2025-02-05 23:19,1970-01-01 0:00,0,0,0,drwxr-xr-x,ubuntu,supergroup
2,/home/ubuntu,0,2025-02-05 23:19,1970-01-01 0:00,0,0,0,drwxr-xr-x,ubuntu,supergroup
3,/user,0,2025-02-05 23:20,1970-01-01 0:00,0,0,0,drwxr-xr-x,ubuntu,supergroup
4,/user/john,0,2025-02-05 23:27,1970-01-01 0:00,0,0,0,drwxr-xr-x,ubuntu,supergroup
5,/user/john/README.txt,1,2025-02-05 23:30,2025-02-06 0:38,134217728,1,350,-rw-r--r--,ubuntu,supergroup


In [70]:
# df["PreferredBlockSize", "BlocksCount","FileSize"].apply(lambda x: x if "." in x.split("/")[-1] else None).tolist()

_df = df.copy()

for i in range(0, len(_df)):
    if "." not in df.loc[i]["Path"].split("/")[-1]:
        _df.loc[i, ["PreferredBlockSize", "BlocksCount", "FileSize"]] = None 

_df


,Path,Replication,ModificationTime,AccessTime,PreferredBlockSize,BlocksCount,FileSize,NSQUOTA,DSQUOTA,Permission,UserName,GroupName
0,/,0,2025-02-05 23:20,1970-01-01 0:00,NaN,NaN,NaN,9223372036854775807,-1,drwxr-xr-x,ubuntu,supergroup
1,/home,0,2025-02-05 23:19,1970-01-01 0:00,NaN,NaN,NaN,-1,-1,drwxr-xr-x,ubuntu,supergroup
2,/home/ubuntu,0,2025-02-05 23:19,1970-01-01 0:00,NaN,NaN,NaN,-1,-1,drwxr-xr-x,ubuntu,supergroup
3,/user,0,2025-02-05 23:20,1970-01-01 0:00,NaN,NaN,NaN,-1,-1,drwxr-xr-x,ubuntu,supergroup
4,/user/john,0,2025-02-05 23:27,1970-01-01 0:00,NaN,NaN,NaN,-1,-1,drwxr-xr-x,ubuntu,supergroup
5,/user/john/README.txt,1,2025-02-05 23:30,2025-02-06 0:38,134217728.0,1.0,350.0,0,0,-rw-r--r--,ubuntu,supergroup


In [71]:
with open('./fsimage2402_1.xml', "w") as f:
    f.write("<FileSystemMetadata>\n")
    for i in range(0, len(_df)):
        f.write("\t<File>\n")

        f.write(f"\t\t<Path>{df.loc[i]["Path"]}</Path>\n")
        f.write(f"\t\t<Replication>{df.loc[i]["Replication"]}</Replication>\n")
        f.write(f"\t\t<ModificationTime>{df.loc[i]["ModificationTime"]}</ModificationTime>\n")
        f.write(f"\t\t<AccessTime>{df.loc[i]["AccessTime"]}</AccessTime>\n")
        if "." not in df.loc[i]["Path"].split("/")[-1] and df.loc[i]["PreferredBlockSize"] == 0 and df.loc[i]["BlocksCount"]  == 0 and df.loc[i]["FileSize"]  == 0:
            pass
        else:
            f.write(f"\t\t<PreferredBlockSize>{df.loc[i]["PreferredBlockSize"]}</PreferredBlockSize>\n")
            f.write(f"\t\t<BlocksCount>{df.loc[i]["BlocksCount"]}</BlocksCount>\n")
            f.write(f"\t\t<FileSize>{df.loc[i]["FileSize"]}</FileSize>\n")

        f.write(f"\t\t<Permission>{df.loc[i]["Permission"]}</Permission>\n")
        f.write(f"\t\t<GroupName>{df.loc[i]["GroupName"]}</GroupName>\n")
        f.write(f"\t\t<UserName>{df.loc[i]["UserName"]}</UserName>\n")
        f.write("\t</File>\n")
    f.write("</FileSystemMetadata>\n")
    


In [115]:
# with open('./fsimage2402_1.xml', "w") as f:
#     f.write("<FileSystemMetadata>\n")
#     for i in range(0, len(df)):
#         f.write("\t<File>\n")
#         for j in list(df.columns):
#             if j in ["PreferredBlockSize", "BlocksCount", "FileSize"]:
#                 if "." in df.loc[i]["Path"].split("/")[-1]:
#                     f.write(f"\t\t<{j}>{df.loc[i][j]}</{j}>\n")
#             elif j in ["NSQUOTA", "DSQUOTA"]:
#                 pass
#             else:
#                 f.write(f"\t\t<{j}>{df.loc[i][j]}</{j}>\n")
#         f.write("\t</File>\n")
#     f.write("</FileSystemMetadata>\n")

str_ = ""
str_ += "<FileSystemMetadata>"
for i in range(0, len(df)):
    str_+= "<File>"
    for j in list(df.columns):
        if j in ["PreferredBlockSize", "BlocksCount", "FileSize"]:
            if "." in df.loc[i]["Path"].split("/")[-1]:
                str_+= f"<{j}>{df.loc[i][j]}</{j}>"
        elif j in ["NSQUOTA", "DSQUOTA"]:
            pass
        else:
            str_+= f"\t\t<{j}>{df.loc[i][j]}</{j}>"
    str_ +="\t</File>"
str_ += "</FileSystemMetadata>"

root = etree.fromstring(str_)
xml_output = etree.tostring(root, pretty_print=True).decode()
print(xml_output)



<FileSystemMetadata>
  <File>		<Path>/</Path>		<Replication>0</Replication>		<ModificationTime>2025-02-05 23:20</ModificationTime>		<AccessTime>1970-01-01 0:00</AccessTime>		<Permission>drwxr-xr-x</Permission>		<UserName>ubuntu</UserName>		<GroupName>supergroup</GroupName>	</File>
  <File>		<Path>/home</Path>		<Replication>0</Replication>		<ModificationTime>2025-02-05 23:19</ModificationTime>		<AccessTime>1970-01-01 0:00</AccessTime>		<Permission>drwxr-xr-x</Permission>		<UserName>ubuntu</UserName>		<GroupName>supergroup</GroupName>	</File>
  <File>		<Path>/home/ubuntu</Path>		<Replication>0</Replication>		<ModificationTime>2025-02-05 23:19</ModificationTime>		<AccessTime>1970-01-01 0:00</AccessTime>		<Permission>drwxr-xr-x</Permission>		<UserName>ubuntu</UserName>		<GroupName>supergroup</GroupName>	</File>
  <File>		<Path>/user</Path>		<Replication>0</Replication>		<ModificationTime>2025-02-05 23:20</ModificationTime>		<AccessTime>1970-01-01 0:00</AccessTime>		<Permission>drwxr-xr-x</

In [116]:
xml_output

'<FileSystemMetadata>\n  <File>\t\t<Path>/</Path>\t\t<Replication>0</Replication>\t\t<ModificationTime>2025-02-05 23:20</ModificationTime>\t\t<AccessTime>1970-01-01 0:00</AccessTime>\t\t<Permission>drwxr-xr-x</Permission>\t\t<UserName>ubuntu</UserName>\t\t<GroupName>supergroup</GroupName>\t</File>\n  <File>\t\t<Path>/home</Path>\t\t<Replication>0</Replication>\t\t<ModificationTime>2025-02-05 23:19</ModificationTime>\t\t<AccessTime>1970-01-01 0:00</AccessTime>\t\t<Permission>drwxr-xr-x</Permission>\t\t<UserName>ubuntu</UserName>\t\t<GroupName>supergroup</GroupName>\t</File>\n  <File>\t\t<Path>/home/ubuntu</Path>\t\t<Replication>0</Replication>\t\t<ModificationTime>2025-02-05 23:19</ModificationTime>\t\t<AccessTime>1970-01-01 0:00</AccessTime>\t\t<Permission>drwxr-xr-x</Permission>\t\t<UserName>ubuntu</UserName>\t\t<GroupName>supergroup</GroupName>\t</File>\n  <File>\t\t<Path>/user</Path>\t\t<Replication>0</Replication>\t\t<ModificationTime>2025-02-05 23:20</ModificationTime>\t\t<AccessT

In [ ]:
# _df.to_xml('./fsimage2402.xml')

In [78]:
tree = etree.parse('./fsimage2402_1.xml')

In [79]:
def printf(elems):
    if (isinstance(elems, list)):
        for elem in elems:
            if isinstance(elem, str):
                print(elem)
            else:
                print(etree.tostring(elem).decode('utf-8'))
    else: # just a single element
        print(etree.tostring(elems).decode('utf-8'))

In [138]:
def ls(txt):
    tree = etree.parse('./fsimage2402_1.xml')
    text = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '{txt}']/*/text()")

    root = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '{txt}']/*")
    lst = [etree.tostring(i, pretty_print=True).decode() for i in root]
    element_tag = [i.split(">")[0].split("<")[-1] for i in lst]


    # if len(root)!= 1:

    #     lst = [etree.tostring(i, pretty_print=True).decode() for i in root]
    #     ele = [i.split(">")[0].split("<")[-1] for i in lst]
    #     print(ele)
    
    #     # root_str = etree.tostring(root[0], pretty_print=True).decode()
    #     return lst

    # root_str = etree.tostring(root, pretty_print=True).decode()
    # return root_str
    # return root

    _dict = {x: y for x, y in zip(element_tag, text)}
    if "PreferredBlockSize" not in _dict.keys():
        _dict.update({"Replication": "-", })
    # if "BlocksCount" not in _dict.keys():
    #     _dict.update({"BlocksCount":, })
    if "FileSize" not in _dict.keys():
        _dict.update({"FileSize": "0", })
    return _dict

    
ans = ls('/user/john')
ans


{'Path': '/user/john',
 'Replication': '-',
 'ModificationTime': '2025-02-05 23:27',
 'AccessTime': '1970-01-01 0:00',
 'Permission': 'drwxr-xr-x',
 'UserName': 'ubuntu',
 'GroupName': 'supergroup',
 'FileSize': '0'}

In [152]:
path = '/user/john/README.txt'
# /john/README.txt

def ls(path):
    tree = etree.parse('./fsimage2402_1.xml')

    if "." in path.split("/")[-1]:
        text = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '{path}']/*/text()")
        root = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '{path}']/*")
        lst = [etree.tostring(i, pretty_print=True).decode() for i in root]
        element_tag = [i.split(">")[0].split("<")[-1] for i in lst]
        _dict = {x: y for x, y in zip(element_tag, text)}
        # print(_dict)
        return [_dict]

    else:
        ls_dict_lst = []
        len_path = len(path.split("/"))
        path_text_lst = tree.xpath(f"/FileSystemMetadata/File[starts-with(Path/text(), '{path}')]/Path/text()")
        immediate_children_lst = [i for i in path_text_lst if len(i.split("/")) == len_path+1]


        # print(immediate_children_lst)
        for p in immediate_children_lst:
            text = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '{p}']/*/text()")

            root = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '{p}']/*")
            lst = [etree.tostring(i, pretty_print=True).decode() for i in root]
            element_tag = [i.split(">")[0].split("<")[-1] for i in lst]
            _dict = {x: y for x, y in zip(element_tag, text)}
            # print(_dict)

            if "." not in path.split("/")[-1]:
                _dict.update({"Replication": "-" })
                _dict.update({"FileSize": "0", })

            ls_dict_lst.append(_dict)
            
        
        return ls_dict_lst

ans = ls(path)
ans

[{'Path': '/user/john/README.txt',
  'Replication': '1',
  'ModificationTime': '2025-02-05 23:30',
  'AccessTime': '2025-02-06 0:38',
  'PreferredBlockSize': '134217728',
  'BlocksCount': '1',
  'FileSize': '350',
  'Permission': '-rw-r--r--',
  'UserName': 'ubuntu',
  'GroupName': 'supergroup'}]

In [113]:
# tree.xpath(f"/FileSystemMetadata/File[Path/text()='/user/john/README.txt']/text()")
file_element = tree.xpath(f"/FileSystemMetadata/File[Path/text() = '/user/john/README.txt']")[0]
etree.tostring(file_element, pretty_print=True).decode()

'<File>\n\t\t<Path>/user/john/README.txt</Path>\n\t\t<Replication>1</Replication>\n\t\t<ModificationTime>2025-02-05 23:30</ModificationTime>\n\t\t<AccessTime>2025-02-06 0:38</AccessTime>\n\t\t<PreferredBlockSize>134217728</PreferredBlockSize>\n\t\t<BlocksCount>1</BlocksCount>\n\t\t<FileSize>350</FileSize>\n\t\t<Permission>-rw-r--r--</Permission>\n\t\t<UserName>ubuntu</UserName>\n\t\t<GroupName>supergroup</GroupName>\n\t</File>\n\n'

In [157]:
[a for a in "abn"]

['a', 'b', 'n']